In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.neighbors import NearestNeighbors
from bramm_data_analysis import preprocessing
import matplotlib.pyplot as plt

# Load Data

### Load Moss Data

In [ ]:
moss_data_path = Path("../data/Mines_2024.xlsx")
df_moss = preprocessing.MossPreprocessor(moss_data_path).load_preprocess()

### Load RMQS Data

In [ ]:
rmqs_data_path = Path("../data/RMQS.csv")
df_rmqs = preprocessing.RMQSPreprocessor(rmqs_data_path).load_preprocess()

In [ ]:
df_rmqs["date_complete"].dt.year.hist()

In [ ]:
year_threshold = 2000
df_rmqs = df_rmqs[df_rmqs["date_complete"].dt.year >= year_threshold]

## Closest Points Matching

In [ ]:
estimator = NearestNeighbors(n_neighbors=1, metric="haversine")
moss_xy = df_moss[["longitude", "latitude"]]
rmqs_xy = df_rmqs[["longitude", "latitude"]]
estimator.fit(rmqs_xy)
dist, indexes = estimator.kneighbors(moss_xy)

nearests = df_rmqs.loc[df_rmqs.index[indexes.flatten()]]
nearests.index = df_moss.index

raw_match = pd.merge(
    left=df_moss,
    right=nearests,
    left_index=True,
    right_index=True,
    suffixes=("_moss", "_rmqs"),
).merge(
    right=pd.DataFrame({"distance": dist.flatten()}, index=nearests.index),
    left_index=True,
    right_index=True,
)


threshold = 0.01
final_match = raw_match[raw_match["distance"] <= threshold].reset_index()
print(
    f"Threshold : {threshold * 6371} km \nConserved : {final_match.shape[0]} / {raw_match.shape[0]}"
)

In [ ]:
# gdf_moss = gpd.GeoDataFrame(
#     data=df_moss,
#     geometry=gpd.points_from_xy(
#         x=df_moss["longitude"], y=df_moss["latitude"], crs="epsg:4326"
#     ),
# )
# gdf_moss_match = gpd.GeoDataFrame(
#     data=df_moss_match,
#     geometry=gpd.points_from_xy(
#         x=df_moss_match["longitude"], y=df_moss_match["latitude"], crs="epsg:4326"
#     ),
# )
# gdf_rmqs_match = gpd.GeoDataFrame(
#     data=rmqs_match,
#     geometry=gpd.points_from_xy(
#         x=rmqs_match["longitude"], y=rmqs_match["latitude"], crs="epsg:4326"
#     ),
# )
# gdf_rmqs_not_match = gpd.GeoDataFrame(
#     data=rmqs_not_match,
#     geometry=gpd.points_from_xy(
#         x=rmqs_not_match["longitude"], y=rmqs_not_match["latitude"], crs="epsg:4326"
#     ),
# )
# gdf_rmqs = gpd.GeoDataFrame(
#     data=df_rmqs,
#     geometry=gpd.points_from_xy(
#         x=df_rmqs["longitude"], y=df_rmqs["latitude"], crs="epsg:4326"
#     ),
# )

In [ ]:
plt.scatter(
    df_moss["longitude"], df_moss["latitude"], label="all moss", alpha=0.4
)
plt.scatter(
    final_match["longitude_moss"],
    final_match["latitude_moss"],
    label="Moss match",
    alpha=0.5,
    color="green",
)
plt.scatter(
    final_match["longitude_rmqs"],
    final_match["latitude_rmqs"],
    label="RMQS match",
    alpha=0.5,
    color="red",
)
plt.legend()
plt.plot()

## Multivariate Analysis

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import gstools as gs

verbose = True
graphics = True
gl.OptCst.define(gl.ECst.NTCOL, 6)
gdoc.setNoScroll()

In [ ]:
print(df_moss.columns)

In [ ]:
print(df_rmqs.filter(like="_tot_hf").columns)

In [ ]:
print(df_moss.columns)

In [ ]:
x1 = "longitude"
x2 = "latitude"
z1 = "aluminium"
z2 = "al_tot_hf"
raw_df = final_match.filter([x1, x2, z1, z2]).astype("float")
final_df = raw_df  # [
# (raw_df[z1] <= 15000)
# &
# (raw_df[z2] <= 1)
# ]

In [ ]:
mydb = gl.Db_fromPanda(final_df)
mydb.setLocators([x1, x2], gl.ELoc.X)
mydb.setLocator(z1, gl.ELoc.Z)

In [ ]:
mydb

In [ ]:
gl.dbStatisticsPrint(
    db=mydb,
    names=[z1, z2],
    opers=gl.EStatOption.fromKeys(["MEAN", "MINI", "MAXI"]),
    flagIso=False,
    title="Number of observations",
)

In [ ]:
ax = gp.correlation(mydb, namex=z1, namey=z2, asPoint=True, regrLine=True)
ax.decoration(title=f"Correlation between {z1} and {z2}")